In [33]:
from tqdm import tqdm
import requests
import time
from zipfile import ZipFile
import os
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [34]:
def download(url, dir):
    time.sleep(5)
    chunk_size= 1024
    r = requests.get(url, stream = True)
    total_size = int(r.headers['content-length'])
    filename =url.split('/')[-1]
    with open(dir+filename, 'wb') as f:
        for data in tqdm(iterable= r.iter_content(chunk_size = chunk_size), total = total_size/chunk_size, unit = 'KB'):
            f.write(data)

In [35]:
entidades=["Aguascalientes:01","Baja California:02","Baja California Sur:03","Campeche:04","Coahuila de Zaragoza:05","Colima:06","Chiapas:07","Chihuahua:08","CDMX:09","Durango:10","Guanajuato:11","Guerrero:12","Hidalgo:13","Jalisco:14","México:15","Michoacán de Ocampo:16","Morelos:17","Nayarit:18","Nuevo León:19","Oaxaca:20","Puebla:21","Querétaro:22","Quintana Roo:23","San Luis Potosí:24","Sinaloa:25","Sonora:26","Tabasco:27","Tamaulipas:28","Tlaxcala:29","Veracruz de Ignacio de la Llave:30","Yucatán:31","Zacatecas:32"]
for ent in entidades:
    print(ent)

Aguascalientes:01
Baja California:02
Baja California Sur:03
Campeche:04
Coahuila de Zaragoza:05
Colima:06
Chiapas:07
Chihuahua:08
CDMX:09
Durango:10
Guanajuato:11
Guerrero:12
Hidalgo:13
Jalisco:14
México:15
Michoacán de Ocampo:16
Morelos:17
Nayarit:18
Nuevo León:19
Oaxaca:20
Puebla:21
Querétaro:22
Quintana Roo:23
San Luis Potosí:24
Sinaloa:25
Sonora:26
Tabasco:27
Tamaulipas:28
Tlaxcala:29
Veracruz de Ignacio de la Llave:30
Yucatán:31
Zacatecas:32


In [36]:
cve_ent=str(input("Escribe la clave del Estado que desea descargar: "))

Escribe la clave del Estado que desea descargar: 03


In [37]:
directory= "./iters/"
iter = f"https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/iter/iter_{cve_ent}_2020_csv.zip"
download(iter,directory)
print("Descarga terminada")

 94%|███████████████████████████████████████████████████████████████▊    | 191/203.5361328125 [00:00<00:00, 395.87KB/s]C:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\std.py:533: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|████████████████████████████████████████████████████████████████████| 204/203.5361328125 [00:00<00:00, 313.05KB/s]

Descarga terminada


In [38]:
zip_file = f"./iters/iter_{cve_ent}_2020_csv.zip"#en esta carpeta esta el zip

#ITER_21CSV20.csv
csv_file = f"ITER_{cve_ent}CSV20.csv"#este va a ser el nombre del archivo csv

with ZipFile(zip_file, 'r') as zip:
    zipInfo = zip.getinfo(csv_file)
    zipInfo.filename = os.path.basename(csv_file)
    zip.extract(zipInfo, directory)

In [39]:
#borrarmos el archivo zip
os.remove(zip_file)

In [40]:
iter20 = pd.read_csv("./iters/"+csv_file)

In [41]:
locsW0 =iter20.loc[:, 'MUN'] != 0 #Le decimos que elig las localidades whitout zero, el tipo de dato es una serie de pandas
locsW0

0       False
1       False
2       False
3        True
4        True
        ...  
2556     True
2557     True
2558     True
2559     True
2560     True
Name: MUN, Length: 2561, dtype: bool

In [42]:
df_localidadesW0 = iter20.loc[locsW0]# creamos un nuevo dataframe con la matriz de booleanos

In [43]:
#Volvemos a crear una matriz de booleanos
locsW02 =df_localidadesW0.loc[:, 'LOC'] != 0
locsW02

3       False
4        True
5        True
6        True
7        True
        ...  
2556     True
2557     True
2558     True
2559     True
2560     True
Name: LOC, Length: 2558, dtype: bool

In [44]:
#creamos un nuevo dataframe con la matriz de booleanos
localidadesOK=df_localidadesW0[locsW02]

In [45]:
#Quitamos datos con clave 9998
#creamos una matriz de booleanos
locsW03 =localidadesOK.loc[:, 'LOC'] != 9998 

In [46]:
#creamos otro dataframe con la matriz de booleanos
localidadesOK2=localidadesOK.loc[locsW03]

In [47]:
#extraemos los grados de la coordenada X en una nueva columnna
localidadesOK2=localidadesOK2.assign(gradosX=localidadesOK2['LONGITUD'].str[0:3])

In [48]:
#extraemos los minutos de la coordenada X en una nueva columnna
localidadesOK2=localidadesOK2.assign(minutosX=localidadesOK2['LONGITUD'].str[4:6])

In [49]:
#extraemos los minutos de la coordenada X en una nueva columnna
localidadesOK2=localidadesOK2.assign(segundosX=localidadesOK2['LONGITUD'].str[7:13])

In [50]:
localidadesOK2["gradosX"]=localidadesOK2["gradosX"].astype(float)
localidadesOK2["minutosX"]=localidadesOK2["minutosX"].astype(float)/60
localidadesOK2["segundosX"]=localidadesOK2["segundosX"].astype(float)/3600

In [51]:
#Sumamos los grados minutos y segundos y los guardarmos en una nueva columna
localidadesOK2=localidadesOK2.assign(X= (localidadesOK2["gradosX"] + localidadesOK2["minutosX"] + localidadesOK2["segundosX"])*-1)

In [52]:
localidadesOK2=localidadesOK2.assign(gradosY=localidadesOK2['LATITUD'].str[0:2])
localidadesOK2=localidadesOK2.assign(minutosY=localidadesOK2['LATITUD'].str[3:5])
localidadesOK2=localidadesOK2.assign(segundosY=localidadesOK2['LATITUD'].str[6:12])

In [53]:
localidadesOK2["gradosY"]=localidadesOK2["gradosY"].astype(float)
localidadesOK2["minutosY"]=localidadesOK2["minutosY"].astype(float)/60
localidadesOK2["segundosY"]=localidadesOK2["segundosY"].astype(float)/3600

In [54]:
localidadesOK2=localidadesOK2.assign(Y= localidadesOK2["gradosY"] + localidadesOK2["minutosY"] + localidadesOK2["segundosY"])

In [55]:
localidadesOK2.to_csv(f"./iters/Localidades{cve_ent}.csv")

In [56]:
#borrarmos el archivo csv que descargamos y conservamos el nuestro
os.remove(f"./iters/{csv_file}")
print("El proceso termino correctamente")

El proceso termino correctamente


In [57]:
localidadesOK2

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_SINTIC,TAMLOC,gradosX,minutosX,segundosX,X,gradosY,minutosY,segundosY,Y
4,3,Baja California Sur,1,Comondú,1,Ciudad Constitución,"111°40'24.022"" W","25°02'04.688"" N",49.0,43805,...,101,9,111.0,0.666667,0.006673,-111.673339,25.0,0.033333,0.001302,25.034636
5,3,Baja California Sur,1,Comondú,3,Puerto Adolfo López Mateos,"112°06'38.209"" W","25°11'34.462"" N",2.0,2227,...,10,4,112.0,0.100000,0.010614,-112.110614,25.0,0.183333,0.009573,25.192906
6,3,Baja California Sur,1,Comondú,20,Andachires,"111°30'22.921"" W","25°23'07.280"" N",110.0,1,...,*,1,111.0,0.500000,0.006367,-111.506367,25.0,0.383333,0.002022,25.385356
7,3,Baja California Sur,1,Comondú,27,Puerto Magdalena,"112°08'23.545"" W","24°38'04.026"" N",3.0,55,...,2,1,112.0,0.133333,0.006540,-112.139874,24.0,0.633333,0.001118,24.634452
8,3,Baja California Sur,1,Comondú,31,Batequitos,"111°21'24.066"" W","25°08'31.554"" N",140.0,14,...,0,1,111.0,0.350000,0.006685,-111.356685,25.0,0.133333,0.008765,25.142098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,3,Baja California Sur,9,Loreto,384,La Herradura,"111°31'41.795"" W","26°18'35.782"" N",54.0,2,...,*,1,111.0,0.516667,0.011610,-111.528276,26.0,0.300000,0.009939,26.309939
2556,3,Baja California Sur,9,Loreto,385,Punta Ballena,"111°19'08.622"" W","26°04'22.590"" N",5.0,2,...,*,1,111.0,0.316667,0.002395,-111.319062,26.0,0.066667,0.006275,26.072942
2557,3,Baja California Sur,9,Loreto,388,La Higuerita,"111°36'49.015"" W","26°29'15.479"" N",97.0,22,...,1,1,111.0,0.600000,0.013615,-111.613615,26.0,0.483333,0.004300,26.487633
2558,3,Baja California Sur,9,Loreto,389,Tecoragui,"111°20'26.785"" W","26°04'32.421"" N",33.0,10,...,N/D,1,111.0,0.333333,0.007440,-111.340774,26.0,0.066667,0.009006,26.075672


In [58]:
df_sin_nan = localidadesOK2.dropna(subset=['X', 'Y'])
df_sin_nan

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_SINTIC,TAMLOC,gradosX,minutosX,segundosX,X,gradosY,minutosY,segundosY,Y
4,3,Baja California Sur,1,Comondú,1,Ciudad Constitución,"111°40'24.022"" W","25°02'04.688"" N",49.0,43805,...,101,9,111.0,0.666667,0.006673,-111.673339,25.0,0.033333,0.001302,25.034636
5,3,Baja California Sur,1,Comondú,3,Puerto Adolfo López Mateos,"112°06'38.209"" W","25°11'34.462"" N",2.0,2227,...,10,4,112.0,0.100000,0.010614,-112.110614,25.0,0.183333,0.009573,25.192906
6,3,Baja California Sur,1,Comondú,20,Andachires,"111°30'22.921"" W","25°23'07.280"" N",110.0,1,...,*,1,111.0,0.500000,0.006367,-111.506367,25.0,0.383333,0.002022,25.385356
7,3,Baja California Sur,1,Comondú,27,Puerto Magdalena,"112°08'23.545"" W","24°38'04.026"" N",3.0,55,...,2,1,112.0,0.133333,0.006540,-112.139874,24.0,0.633333,0.001118,24.634452
8,3,Baja California Sur,1,Comondú,31,Batequitos,"111°21'24.066"" W","25°08'31.554"" N",140.0,14,...,0,1,111.0,0.350000,0.006685,-111.356685,25.0,0.133333,0.008765,25.142098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,3,Baja California Sur,9,Loreto,383,La Pitaya,"111°22'20.770"" W","26°03'47.821"" N",103.0,2,...,*,1,111.0,0.366667,0.005769,-111.372436,26.0,0.050000,0.013284,26.063284
2555,3,Baja California Sur,9,Loreto,384,La Herradura,"111°31'41.795"" W","26°18'35.782"" N",54.0,2,...,*,1,111.0,0.516667,0.011610,-111.528276,26.0,0.300000,0.009939,26.309939
2556,3,Baja California Sur,9,Loreto,385,Punta Ballena,"111°19'08.622"" W","26°04'22.590"" N",5.0,2,...,*,1,111.0,0.316667,0.002395,-111.319062,26.0,0.066667,0.006275,26.072942
2557,3,Baja California Sur,9,Loreto,388,La Higuerita,"111°36'49.015"" W","26°29'15.479"" N",97.0,22,...,1,1,111.0,0.600000,0.013615,-111.613615,26.0,0.483333,0.004300,26.487633


In [59]:
x=df_sin_nan['X'].iloc[0]
y=df_sin_nan['Y'].iloc[0]
print(str(x), str(y))

-111.67333944444445 25.034635555555557


In [60]:
mapa = folium.Map(location=[y, x], zoom_start=9) # Creamos un mapa centrado en una ubicación específica (usando las coordenadas del primer punto de tu DataFrame)
marker_cluster = MarkerCluster().add_to(mapa)
marker_cluster

In [61]:
for index, localidad in df_sin_nan.iterrows():
    X=float(localidad['X'])
    Y=float(localidad['Y'])    
    folium.Marker(location=[Y,X], popup=localidad['NOM_LOC'],  icon=folium.Icon(color="green", icon="ok-sign"),).add_to(marker_cluster)

In [ ]:
mapa

In [ ]:
mapa.save("mapa.html")